Data Analysis with PyGazeAnalyzer 1.0

In [25]:
import numpy


def remove_missing(x, y, time, missing):
	mx = numpy.array(x==missing, dtype=int)
	my = numpy.array(y==missing, dtype=int)
	x = x[(mx+my) != 2]
	y = y[(mx+my) != 2]
	time = time[(mx+my) != 2]
	return x.reset_index(drop=True), y.reset_index(drop=True), time.reset_index(drop=True)


def fixation_detection_fixed(x, y, time, missing=0.0, maxdist=25, mindur=50):
	
	"""Detects fixations, defined as consecutive samples with an inter-sample
	distance of less than a set amount of pixels (disregarding missing data)
	
	arguments

	x		-	numpy array of x positions
	y		-	numpy array of y positions
	time		-	numpy array of EyeTribe timestamps

	keyword arguments

	missing	-	value to be used for missing data (default = 0.0)
	maxdist	-	maximal inter sample distance in pixels (default = 25)
	mindur	-	minimal duration of a fixation in milliseconds; detected
				fixation cadidates will be disregarded if they are below
				this duration (default = 100)
	
	returns
	Sfix, Efix
				Sfix	-	list of lists, each containing [starttime]
				Efix	-	list of lists, each containing [starttime, endtime, duration, endx, endy]
	"""
	#print("fixation_detection_fixed", x, y, time, missing, maxdist, mindur)
    
	x, y, time = remove_missing(x, y, time, missing)
    
	#print("remove_missing",x,y, time)

	# empty list to contain data
	Sfix = []
	Efix = []
	
	# loop through all coordinates
	si = 0
	fixstart = False
	for i in range(1,len(x)):
		# calculate Euclidean distance from the current fixation coordinate
		# to the next coordinate
		squared_distance = ((x[si]-x[i])**2 + (y[si]-y[i])**2)
		dist = 0.0

		if squared_distance > 0:
			dist = squared_distance**0.5
            
		#print("loop", i, squared_distance, dist)
        
		# check if the next coordinate is below maximal distance
		if dist <= maxdist and not fixstart:
			# start a new fixation
			si = 0 + i
			fixstart = True
			Sfix.append([time[i]])
		elif dist > maxdist and fixstart:
			# end the current fixation
			fixstart = False
			# only store the fixation if the duration is ok
			if time[i-1]-Sfix[-1][0] >= mindur:
				Efix.append([Sfix[-1][0], time[i-1], time[i-1]-Sfix[-1][0], x[si], y[si]])
			# delete the last fixation start if it was too short
			else:
				Sfix.pop(-1)
			si = 0 + i
		elif not fixstart:
			si += 1
	#add last fixation end (we can lose it if dist > maxdist is false for the last point)
	if len(Sfix) > len(Efix):
		Efix.append([Sfix[-1][0], time[len(x)-1], time[len(x)-1]-Sfix[-1][0], x[si], y[si]])
	return Sfix, Efix


Saccade Metrics

In [38]:

def saccade_detection_fixed(x, y, time, missing=0.0, minlen=5, maxvel=40, maxacc=340):
	
	"""Detects saccades, defined as consecutive samples with an inter-sample
	velocity of over a velocity threshold or an acceleration threshold
	
	arguments

	x		-	numpy array of x positions
	y		-	numpy array of y positions
	time		-	numpy array of tracker timestamps in milliseconds

	keyword arguments

	missing	-	value to be used for missing data (default = 0.0)
	minlen	-	minimal length of saccades in milliseconds; all detected
				saccades with len(sac) < minlen will be ignored
				(default = 5)
	maxvel	-	velocity threshold in pixels/second (default = 40)
	maxacc	-	acceleration threshold in pixels / second**2
				(default = 340)
	
	returns
	Ssac, Esac
			Ssac	-	list of lists, each containing [starttime]
			Esac	-	list of lists, each containing [starttime, endtime, duration, startx, starty, endx, endy]
	"""
	x, y, time = remove_missing(x, y, time, missing)

	# CONTAINERS
	Ssac = []
	Esac = []

	# INTER-SAMPLE MEASURES
	# the distance between samples is the square root of the sum
	# of the squared horizontal and vertical interdistances
	intdist = (numpy.diff(x)**2 + numpy.diff(y)**2)**0.5
	# get inter-sample times
	inttime = numpy.diff(time)

	# recalculate inter-sample times to seconds
	inttime = inttime / 1000.0
	
	# VELOCITY AND ACCELERATION
	# the velocity between samples is the inter-sample distance
	# divided by the inter-sample time
	vel = intdist / inttime
	# the acceleration is the sample-to-sample difference in
	# eye movement velocity
	acc = numpy.diff(vel)

	# SACCADE START AND END
	t0i = 0
	stop = False
	while not stop:
		# saccade start (t1) is when the velocity or acceleration
		# surpass threshold, saccade end (t2) is when both return
		# under threshold
	
		# detect saccade starts
		sacstarts = numpy.where((vel[1+t0i:] > maxvel).astype(int) + (acc[t0i:] > maxacc).astype(int) >= 1)[0]
		if len(sacstarts) > 0:
			# timestamp for starting position
			t1i = t0i + sacstarts[0] + 1
			if t1i >= len(time)-1:
				t1i = len(time)-2
			t1 = time[t1i]
			
			# add to saccade starts
			Ssac.append([t1])
			
			# detect saccade endings
			sacends = numpy.where((vel[1+t1i:] < maxvel).astype(int) + (acc[t1i:] < maxacc).astype(int) == 2)[0]
			if len(sacends) > 0:
				# timestamp for ending position
				t2i = sacends[0] + 1 + t1i + 2
				if t2i >= len(time):
					t2i = len(time)-1
				t2 = time[t2i]
				dur = t2 - t1

				# ignore saccades that did not last long enough
				if dur >= minlen:
					# add to saccade ends
					Esac.append([t1, t2, dur, x[t1i], y[t1i], x[t2i], y[t2i]])
				else:
					# remove last saccade start on too low duration
					Ssac.pop(-1)

				# update t0i
				t0i = 0 + t2i
			else:
				stop = True
		else:
			stop = True
	
	return Ssac, Esac


In [35]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

def call_fixation_detection_on_data(fixation_info, participant, time, x, y, task = 0):
	missing = 0.0  # Specify the missing value threshold (if any)
	maxdist = 25  # Maximum distance for a fixation (adjust as needed)
	mindur = 50  # Minimum duration for a fixation (adjust as needed)

	# Perform fixation detection using the fixed fixation_detection function
	Sfix, Efix = fixation_detection_fixed(x, y, time, missing=missing, maxdist=maxdist, mindur=mindur)

	# Calculate total fixation duration and average fixation duration
	total_duration = sum(sublist[2] for sublist in Efix)
	average_duration = total_duration / len(Efix) if len(Efix) > 0 else 0

	# Update the fixation_info dictionary
	fixation_info['Participant'].append(participant)
	fixation_info['Task'].append(task)
	fixation_info['Fixation Count'].append(len(Efix))
	fixation_info['Total Fixation Duration [ms]'].append(total_duration)
	fixation_info['Average Fixation Duration [ms]'].append(average_duration)

def call_saccade_detection_on_data(saccade_info, participant, time, x, y, task = 0):
    missing = 0.0  # Specify the missing value threshold (if any)
    minlen = 5  # Maximum distance for a saccade (adjust as needed)
    maxvel = 40  # Minimum duration for a saccade (adjust as needed)
    maxacc = 340

    # Perform saccade detection using the saccade_detection function
    Ssac, Esac = saccade_detection_fixed(x, y, time, missing=missing, minlen=minlen, maxvel=maxvel, maxacc=maxacc)

    # Calculate total saccade duration and average saccade duration
    total_duration = sum(sublist[2] for sublist in Esac)
    average_duration = total_duration / len(Esac) if len(Esac) > 0 else 0

    # Update the saccade_info dictionary
    saccade_info['Participant'].append(participant)
    saccade_info['Task'].append(task)
    saccade_info['Saccade Count'].append(len(Esac))
    saccade_info['Total Saccade Duration [ms]'].append(total_duration)
    saccade_info['Average Saccade Duration [ms]'].append(average_duration)



def prepare_tobii_data(directory_path, file_name, fixation_info, fn):
	tsv_file = os.path.join(directory_path, file_name)

	# Load the Tobii eye tracker data into a Pandas DataFrame
	df = pd.read_csv(tsv_file, delimiter='\t', low_memory=False)
	df = df[['Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Recording timestamp [ms]']]
	df = df.fillna(0.0)

	# Define parameters for fixation detection
	x = df['Gaze point X [DACS px]']  # X-coordinate data
	y = df['Gaze point Y [DACS px]']  # Y-coordinate data
	time = df['Recording timestamp [ms]']

	fn(fixation_info, os.path.splitext(file_name)[0], time, x, y)

def prepare_csv_data(directory_path, file_name, fixation_info, fn):
	csv_file = os.path.join(directory_path, file_name)

	file_name_split = file_name.split('-')
	task_number = file_name_split[0][1:]
	participant_id = file_name_split[1]

	# Load the eye tracker data into a Pandas DataFrame
	df = pd.read_csv(csv_file, delimiter=',', low_memory=False, on_bad_lines='skip', dtype = {'x': float, 'y': str, 'time': float})
	time_column = df.filter(like='TIME(')
	df = df[['FPOGX', 'FPOGY']]

	time_column = time_column.rename(columns={time_column.columns[0]: 'TIME'})

	# Add the renamed column to the original DataFrame
	df = pd.concat([df, time_column], axis=1)

	df = df.fillna(0.0)
	df['FPOGX'] = df['FPOGX'] * 1920
	df['FPOGY'] = df['FPOGY'] * 1080


	# convert time to milliseconds
	df['TIME'] = df['TIME']*1000


	# Call the defined fn
	fn(fixation_info, participant_id, df['TIME'], df['FPOGX'], df['FPOGY'], task_number)


def prepare_xml_data(directory_path, file_name, fixation_info, fn):
	# We only want the eclipse xml files as those have the x and y coordinates
	if not 'eclipse' in file_name:
		return

	xml_file = os.path.join(directory_path, file_name)

	path_elements = directory_path.split(os.sep)
	participant_id = path_elements[-3]
	task = path_elements[-2].split('-')[2]

	tree = ET.parse(xml_file)
	root = tree.getroot()

	# Extract data from XML
	data = []
	for response_elem in root.findall(".//response"):
		x = float(response_elem.get("x"))
		y = float(response_elem.get("y"))
		# Event time is in nanoseconds, so we divide by 1000000 to get milliseconds
		event_time = int(response_elem.get("event_time")) / 1000000
		
		data.append({"event_time": event_time, "x": x, "y": y})

	# Create DataFrame
	df = pd.DataFrame(data)

	fn(fixation_info, participant_id, df['event_time'], df['x'], df['y'], task)


def prepare_txt_data(directory_path, file_name, fixation_info, fn):
	# We only want the ogama txt files as those have the x and y coordinates
	if not 'ogama' in file_name:
		return
	txt_file = os.path.join(directory_path, file_name)

	path_elements = directory_path.split(os.sep)
	participant_id = path_elements[-1]

	# Load the eye tracker data into a Pandas DataFrame
	df = pd.read_csv(txt_file, delimiter=',', low_memory=False, on_bad_lines='skip', encoding = "utf-16")
	df = df[[' ImageName', ' X', ' Y', ' StartTime', ' Included?', ' StimulusType']]

	# Remove all rows where Included? == N
	df = df[df[' Included?'] == 'Y']

	df = df.fillna(0.0)

	
	# Iterate over all unique images
	for image_name in df[' ImageName'].unique():
		# Get the data for the current image
		image_df = df[df[' ImageName'] == image_name]

		# Call the defined fn
		fn(fixation_info, participant_id, image_df[' StartTime'], image_df[' X'], image_df[' Y'], image_name)
	

ending_to_function = {
    '.tsv': prepare_tobii_data,
    '.csv': prepare_csv_data,
	'.xml': prepare_xml_data,
	'.txt': prepare_txt_data
}

test_fixation_info = {
        'Participant': [],
        'Task': [],
        'Fixation Count': [],
        'Total Fixation Duration [ms]': [],
        'Average Fixation Duration [ms]': []
    }



#prepare_txt_data('data\\4\\formatted-raw-data\\151\\', 'ogama.txt', test_fixation_info, call_fixation_detection_on_data)
#print(pd.DataFrame(test_fixation_info))

In [3]:
import os
import pandas as pd


def fixation_data_analysis(directory_path, output_csv = "pygaze_fixations.csv"):

    # Initialize a dictionary to store the fixation counts, total fixation duration, and average fixation duration for each file
    fixation_info = {
        'Participant': [],
        'Task': [],
        'Fixation Count': [],
        'Total Fixation Duration [ms]': [],
        'Average Fixation Duration [ms]': []
    }


    # List all files in the directory and subfolders
    file_names = {}
    for root, dirs, files in os.walk(directory_path):
        dir_filenames = [file_name for file_name in files]
        file_names[root] = dir_filenames

    # Iterate over the files in the directory
    for directory, file_names in file_names.items():
        for file_name in file_names:
            ending = os.path.splitext(file_name)[1]
            if ending in ending_to_function:
                ending_to_function[ending](directory, file_name, fixation_info, call_fixation_detection_on_data)

    # Create a DataFrame to store the fixation information
    count_df = pd.DataFrame(fixation_info)

    # Define the output CSV file path
    output_csv = os.path.join("results/", output_csv)

    # Write the DataFrame to a CSV file
    count_df.to_csv(output_csv, index=False)

    # Print the fixation information
    print(count_df)

    print(f"Fixation information saved to {output_csv}")
  


In [4]:
import os
import pandas as pd


def call_saccade_detection_on_data(saccade_info, participant, time, x, y, task = 0):
    missing = 0.0  # Specify the missing value threshold (if any)
    minlen = 5  # Maximum distance for a saccade (adjust as needed)
    maxvel = 40  # Minimum duration for a saccade (adjust as needed)
    maxacc = 340

    # Perform saccade detection using the saccade_detection function
    Ssac, Esac = saccade_detection_fixed(x, y, time, missing=missing, minlen=minlen, maxvel=maxvel, maxacc=maxacc)

    # Calculate total saccade duration and average saccade duration
    total_duration = sum(sublist[2] for sublist in Esac)
    average_duration = total_duration / len(Esac) if len(Esac) > 0 else 0

    # Update the saccade_info dictionary
    saccade_info['Participant'].append(participant)
    saccade_info['Task'].append(task)
    saccade_info['Saccade Count'].append(len(Esac))
    saccade_info['Total Saccade Duration [ms]'].append(total_duration)
    saccade_info['Average Saccade Duration [ms]'].append(average_duration)


def prepare_tobii_data_saccades(directory_path, file_name, saccade_info):
	tsv_file = os.path.join(directory_path, file_name)

	# Load the Tobii eye tracker data into a Pandas DataFrame
	df = pd.read_csv(tsv_file, delimiter='\t', low_memory=False)
	df = dataframe['Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Recording timestamp [ms]']
	df = df.fillna(0.0)

	# Define parameters for saccade detection
	x = df['Gaze point X [DACS px]']  # X-coordinate data
	y = df['Gaze point Y [DACS px]']  # Y-coordinate data
	time = df['Recording timestamp [ms]']

	call_saccade_detection_on_data(saccade_info, os.path.splitext(file_name)[0], time, x, y)

def prepare_csv_data_saccades(directory_path, file_name, saccade_info):
	csv_file = os.path.join(directory_path, file_name)

	# Load the eye tracker data into a Pandas DataFrame
	df = pd.read_csv(csv_file, delimiter=',', low_memory=False, on_bad_lines='skip', dtype = {'x': float, 'y': str, 'time': float})
	df = df['participant_id', 'x', 'y', 'time', 'task']
	df = df.fillna(0.0)

	# hack because data has wrong value in y column of type string at row 43575
	df = df[pd.to_numeric(df['y'], errors='coerce').notnull()]	
	df['y'] = pd.to_numeric(df['y'])

	# convert time to milliseconds
	#df['time'] = df['time']*1000


	# Call saccade detection on each part of the data where the participant_id is the same
	participant_ids = df['participant_id'].unique()
	for participant_id in participant_ids:
		participant_df = df[df['participant_id'] == participant_id]
		participant_tasks = participant_df['task'].unique()
		for task in participant_tasks:
			task_df = participant_df[participant_df['task'] == task]
			call_saccade_detection_on_data(saccade_info, participant_id, task_df['time'], task_df['x'], task_df['y'], task)


	

In [5]:
import os
import pandas as pd


def saccade_data_analysis(directory_path, output_csv = "pygaze_saccades.csv"):
    """
    Parameters:
        directory_path (str): The path to the directory containing the Tobii eye tracker data TSV files.

    Returns:
        pd.DataFrame: A DataFrame containing the saccade information for each file.
    """
    # Initialize a dictionary to store the saccade counts, total saccade duration, and average saccade duration for each file
    saccade_info = {
        'Participant': [],
        'Task': [],
        'Saccade Count': [],
        'Total Saccade Duration [ms]': [],
        'Average Saccade Duration [ms]': []
    }

    # List all files in the directory and subfolders
    file_names = {}
    for root, dirs, files in os.walk(directory_path):
        dir_filenames = [file_name for file_name in files]
        file_names[root] = dir_filenames

    # Iterate over the files in the directory
    for directory, file_names in file_names.items():
        for file_name in file_names:
            ending = os.path.splitext(file_name)[1]
            if ending in ending_to_function:
                ending_to_function[ending](directory, file_name, saccade_info, call_saccade_detection_on_data)
        
            

    # Create a DataFrame to store the saccade information
    count_df = pd.DataFrame(saccade_info)

    output_csv = os.path.join("results/", output_csv)

    # Write the DataFrame to a CSV file
    count_df.to_csv(output_csv, index=False)

    # Print the saccade information
    print(count_df)

    print(f"Saccade information saved to {output_csv}")




The functions are now declared, we can analyze the data now

In [39]:

#fixation_data_analysis('data\\26\\raw_data\\', "pygaze_fixations_26.csv")
#saccade_data_analysis('data\\26\\raw_data\\', "pygaze_saccades_26.csv")

#fixation_data_analysis('data\\54\\', "pygaze_fixations_54.csv")
#saccade_data_analysis('data\\54\\', "pygaze_saccades_54.csv")

fixation_data_analysis('data\\4\\formatted-raw-data\\', "pygaze_fixations_4.csv")
saccade_data_analysis('data\\4\\formatted-raw-data\\', "pygaze_saccades_4.csv")

     Participant     Task  Saccade Count  Total Saccade Duration [ms]  \
0            151    1.BMP            247                      30441.6   
1            151    2.BMP            131                      11936.5   
2            151   31.BMP             10                      20102.4   
3            151    3.BMP              4                      17700.5   
4            151   32.BMP              7                      10226.7   
...          ...      ...            ...                          ...   
2619         204   28.BMP              2                       7974.1   
2620         204   29.BMP              4                       7273.6   
2621         204   89.BMP              2                       1100.9   
2622         204   30.BMP              3                       6222.5   
2623         204   90.BMP              0                          0.0   

      Average Saccade Duration [ms]  
0                        123.245344  
1                         91.118321  
2        

Analyze the orginal data for fixations/saccades

In [40]:

def analyze_tobii_data(directory_path, file_name, fixation_info):
	tsv_file = os.path.join(directory_path, file_name)

	# Load the Tobii eye tracker data into a Pandas DataFrame
	df = pd.read_csv(tsv_file, delimiter='\t', low_memory=False)
	df = df[['Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Recording timestamp [ms]']]
	df = df.fillna(0.0)

	# Define parameters for fixation detection
	x = df['Gaze point X [DACS px]']  # X-coordinate data
	y = df['Gaze point Y [DACS px]']  # Y-coordinate data
	time = df['Recording timestamp [ms]']

def analyze_csv_data(directory_path, file_name, fixation_info):
	csv_file = os.path.join(directory_path, file_name)

	file_name_split = file_name.split('-')
	task_number = file_name_split[0][1:]
	participant_id = file_name_split[1]

	# Load the eye tracker data into a Pandas DataFrame
	df = pd.read_csv(csv_file, delimiter=',', low_memory=False, on_bad_lines='skip')
	time_column = df.filter(like='TIME(')

	time_column = time_column.rename(columns={time_column.columns[0]: 'TIME'})

	df = time_column

	# convert time to milliseconds
	df['TIME'] = df['TIME']*1000

	total_duration = df['TIME'].max() - df['TIME'].min()
	average_duration = total_duration / len(df['TIME'])

	fixation_info['Participant'].append(participant_id)
	fixation_info['Task'].append(task_number)
	fixation_info['Fixation Count'].append(len(df['TIME']))
	fixation_info['Total Fixation Duration [ms]'].append(total_duration)
	fixation_info['Average Fixation Duration [ms]'].append(average_duration)



def analyze_xml_data(directory_path, file_name, fixation_info):
	# We only want the eclipse xml files as those have the x and y coordinates
	if not file_name.contains('eclipse'):
		return

	xml_file = os.path.join(directory_path, file_name)

	path_elements = directory_path.split(os.sep)
	participant_id = path_elements[-3]
	task = path_elements[-2].split('-')[2]

	tree = ET.parse(xml_file)
	root = tree.getroot()

	# Extract data from XML
	data = []
	for response_elem in root.findall(".//response"):
		x = float(response_elem.get("x"))
		y = float(response_elem.get("y"))
		event_time = int(response_elem.get("event_time"))
		
		data.append({"event_time": event_time, "x": x, "y": y})

	# Create DataFrame
	df = pd.DataFrame(data)

	
original_ending_to_function = {
	'.tsv': analyze_tobii_data,
	'.csv': analyze_csv_data,
	'.xml': analyze_xml_data
}

In [41]:
def original_fixation_data_analysis(directory_path, output_csv = "pygaze_fixations_original.csv"):

    # Initialize a dictionary to store the fixation counts, total fixation duration, and average fixation duration for each file
    fixation_info = {
        'Participant': [],
        'Task': [],
        'Fixation Count': [],
        'Total Fixation Duration [ms]': [],
        'Average Fixation Duration [ms]': []
    }

    saccade_info = {
        'Participant': [],
        'Task': [],
        'Saccade Count': [],
        'Total Saccade Duration [ms]': [],
        'Average Saccade Duration [ms]': []
    }


    # List all files in the directory and subfolders
    file_names = {}
    for root, dirs, files in os.walk(directory_path):
        dir_filenames = [file_name for file_name in files]
        file_names[root] = dir_filenames

    # Iterate over the files in the directory
    print(file_names.items())
    for directory, file_names in file_names.items():
        for file_name in file_names:
            ending = os.path.splitext(file_name)[1]
            if ending in original_ending_to_function:
                original_ending_to_function[ending](directory, file_name, fixation_info)

    # Create a DataFrame to store the fixation information
    count_df = pd.DataFrame(fixation_info)

    # Define the output CSV file path
    output_csv = os.path.join("results/", output_csv)

    # Write the DataFrame to a CSV file
    count_df.to_csv(output_csv, index=False)

    # Print the fixation information
    print(count_df)

    print(f"Fixation information saved to {output_csv}")


dict_items([('data\\26\\fixations\\', ['T0-P108-fixations.csv', 'T0-P164-fixations.csv', 'T0-P165-fixations.csv', 'T0-P169-fixations.csv', 'T0-P193-fixations.csv', 'T0-P228-fixations.csv', 'T0-P309-fixations.csv', 'T0-P313-fixations.csv', 'T0-P320-fixations.csv', 'T0-P322-fixations.csv', 'T0-P327-fixations.csv', 'T0-P340-fixations.csv', 'T0-P370-fixations.csv', 'T0-P372-fixations.csv', 'T0-P376-fixations.csv', 'T0-P379-fixations.csv', 'T0-P382-fixations.csv', 'T0-P393-fixations.csv', 'T0-P431-fixations.csv', 'T0-P435-fixations.csv', 'T0-P442-fixations.csv', 'T0-P459-fixations.csv', 'T0-P490-fixations.csv', 'T0-P523-fixations.csv', 'T0-P539-fixations.csv', 'T0-P548-fixations.csv', 'T0-P561-fixations.csv', 'T0-P604-fixations.csv', 'T0-P606-fixations.csv', 'T0-P620-fixations.csv', 'T0-P623-fixations.csv', 'T0-P627-fixations.csv', 'T0-P637-fixations.csv', 'T0-P641-fixations.csv', 'T0-P709-fixations.csv', 'T0-P718-fixations.csv', 'T0-P722-fixations.csv', 'T0-P742-fixations.csv', 'T0-P745-fi

In [ ]:
original_fixation_data_analysis('data\\26\\fixations\\', "pygaze_fixations_original_26.csv")